### Setup and Imports

This notebook downloads hourly Environment Canada weather data for all Ontario stations for the year 2025.

In [1]:
import pandas as pd
from datetime import datetime
from env_canada import ECHistoricalRange
from io import StringIO
import time
import nest_asyncio
import os

### Load Ontario weather station metadata

A reference table containing station names and ids was previously created by selecting only stations relevant to the agricultural sector in Ontario.

In [2]:
station_ids = pd.read_csv("../data/metadata/weather_stations.csv")
station_ids = station_ids["station_id"].astype(int).tolist()
print(len(station_ids), station_ids[:10])

99 [42967, 52900, 10220, 26799, 42183, 44183, 55238, 53378, 47567, 7868]


### Define helper function for downloading hourly station data


In [3]:
def fetch_station_hourly(station_id, start_date, end_date):
    try:
        ec = ECHistoricalRange(
            station_id=station_id,
            timeframe="hourly",
            daterange=(start_date, end_date),
        )
        ec.get_data()

        df = pd.read_csv(StringIO(ec.csv), sep=";")
        df.columns = df.columns.str.strip()
        df["station_id"] = station_id
        df["Date/Time (LST)"] = pd.to_datetime(df["Date/Time (LST)"], errors="coerce")
        return df

    except Exception as e:
        print(f"❌ Station {station_id} failed: {e}")
        return None

### Initialize environment and define update window  
Apply async support, define the local weather CSV path, set the requested date range (capped to today), and load existing weather data if the file already exists.

In [ ]:
nest_asyncio.apply()

csv_path = "../data/raw/hourly_weather_data.csv"

start_date = datetime(2023, 1, 1)
end_date   = datetime(2025, 12, 31)
end_date   = min(end_date, datetime.now())

# load existing CSV (or start empty)
if os.path.exists(csv_path):
    df_weather = pd.read_csv(csv_path)
    df_weather["Date/Time (LST)"] = pd.to_datetime(df_weather["Date/Time (LST)"], errors="coerce")
else:
    df_weather = pd.DataFrame()

### Identify missing dates per station and download only required data  
For each weather station, compare the dates already present in the existing dataset against the requested date range, identify any missing days, and download only the necessary hourly data to fill those gaps while avoiding redundant requests.


In [ ]:
new_frames = []
3
0

for sid in station_ids:
    # what dates we already have for this station
    if not df_weather.empty:
        have_dates = (
            df_weather.loc[df_weather["station_id"] == sid, "Date/Time (LST)"]
            .dropna()
            .dt.date
            .unique()
        )
        have_dates = set(have_dates)
    else:
        have_dates = set()

    want_dates = set(pd.date_range(start_date, end_date, freq="D").date)
    missing_dates = sorted(want_dates - have_dates)

    if not missing_dates:
        print(f"✅ {sid}: up to date")
        continue

    # download one continuous range: from first missing day to last missing day
    dl_start = datetime.combine(missing_dates[0], datetime.min.time())
    dl_end   = datetime.combine(missing_dates[-1], datetime.max.time())

    print(f"⬇️ {sid}: downloading {missing_dates[0]} to {missing_dates[-1]} ({len(missing_dates)} missing days)")
    df_sid = fetch_station_hourly(sid, dl_start, dl_end)
    if df_sid is not None and not df_sid.empty:
        new_frames.append(df_sid)

    time.sleep(1)

⬇️ 42967: downloading 2023-01-01 to 2024-12-31 (731 missing days)
⬇️ 52900: downloading 2023-01-01 to 2024-12-31 (731 missing days)
⬇️ 10220: downloading 2023-01-01 to 2024-12-31 (731 missing days)
⬇️ 26799: downloading 2023-01-01 to 2024-12-31 (731 missing days)
⬇️ 42183: downloading 2023-01-01 to 2024-12-31 (731 missing days)
⬇️ 44183: downloading 2023-01-01 to 2024-12-31 (731 missing days)
⬇️ 55238: downloading 2023-01-01 to 2024-12-31 (731 missing days)
⬇️ 53378: downloading 2023-01-01 to 2024-12-31 (731 missing days)
⬇️ 47567: downloading 2023-01-01 to 2024-12-31 (731 missing days)
⬇️ 7868: downloading 2023-01-01 to 2024-12-31 (731 missing days)
⬇️ 27674: downloading 2023-01-01 to 2024-12-31 (731 missing days)
⬇️ 55360: downloading 2023-01-01 to 2024-12-31 (731 missing days)
⬇️ 53603: downloading 2023-01-01 to 2024-12-31 (731 missing days)
⬇️ 55258: downloading 2023-01-01 to 2024-12-31 (731 missing days)
⬇️ 52118: downloading 2023-01-01 to 2024-12-31 (731 missing days)
⬇️ 7870: do

### Append new data, remove duplicates, and persist updated dataset  
Combine newly downloaded weather records with the existing dataset, enforce uniqueness at the station–hour level, and save the updated weather file back to disk only if new data was retrieved.

In [6]:
# append + dedupe + save
if new_frames:
    df_new = pd.concat(new_frames, ignore_index=True)
    df_weather = pd.concat([df_weather, df_new], ignore_index=True)

    df_weather["Date/Time (LST)"] = pd.to_datetime(df_weather["Date/Time (LST)"], errors="coerce")
    df_weather = df_weather.drop_duplicates(subset=["station_id", "Date/Time (LST)"]).reset_index(drop=True)

    df_weather.to_csv(csv_path, index=False)
    print("✅ Saved:", csv_path, "| shape:", df_weather.shape)
else:
    print("No new data needed. CSV unchanged.")

✅ Saved: ../data/raw/hourly_weather_data.csv | shape: (2601819, 32)


In [67]:
df_weather.sort_values(
    by=["Date/Time (LST)", "Station Name"],
    ascending=[False, True]
).head()

,Date/Time (LST),Longitude (x),Latitude (y),Station Name,Climate ID,Year,Month,Day,Time (LST),Flag,...,Visibility (km),Visibility Flag,Stn Press (kPa),Stn Press Flag,Hmdx,Hmdx Flag,Wind Chill,Wind Chill Flag,Weather,station_id
8736,2025-12-31,-78.27,45.53,ALGONQUIN PARK EAST GATE,6080192,2025,12,31,00:00,NaN,...,NaN,NaN,95.28,NaN,NaN,NaN,-21.0,NaN,NaN,42967
17473,2025-12-31,-88.91,50.29,ARMSTRONG A,6040327,2025,12,31,00:00,NaN,...,16.1,NaN,97.16,NaN,NaN,NaN,-27.0,NaN,NaN,52900
26210,2025-12-31,-91.63,48.76,ATIKOKAN (AUT),6020LPQ,2025,12,31,00:00,NaN,...,NaN,NaN,96.48,NaN,NaN,NaN,-17.0,NaN,NaN,10220
34947,2025-12-31,-77.88,45.07,BANCROFT AUTO,616I001,2025,12,31,00:00,NaN,...,NaN,NaN,96.20,NaN,NaN,NaN,-20.0,NaN,NaN,26799
43684,2025-12-31,-79.55,44.48,BARRIE-ORO,6117700,2025,12,31,00:00,NaN,...,NaN,NaN,96.95,NaN,NaN,NaN,-15.0,NaN,NaN,42183
